# 🔄 Silver - tb_motivo (SRC)

**Camada**: Silver  
**Catálogo**: v_credit  
**Tabelas de Saída**:
- `tb_motivo` → Registros válidos
- `tb_motivo_invalidos` → Registros rejeitados (auditoria)

**Origem**: `v_credit.bronze.base_motivos`

**Objetivo**: Limpar, padronizar e validar dados de motivos com governança de qualidade

## 📋 Imports e Configuração

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

processing_timestamp = spark.sql("SELECT current_timestamp() as ts").collect()[0]['ts']

In [0]:
CATEGORIAS = ["CADASTRAL", "CARTAO", "FINANCEIRO", "ATENDIMENTO", "BENEFICIOS", "NAO CLASSIFICADO"	]
CRITICIDADE = {
    "BAIXA": "BAIXA",
    "MÉDIA": "MEDIA",
    "MEDIA": "MEDIA",
    "ALTA": "ALTA"
}

## 1️⃣ Leitura da Bronze

In [0]:
TABLE_SOURCE = "v_credit.bronze.base_motivos"
TABLE_TARGET_VALIDOS = "v_credit.silver.tb_motivo"
TABLE_TARGET_INVALIDOS = "v_credit.silver.tb_motivo_invalidos"

In [0]:
df_bronze = spark.table(TABLE_SOURCE)

total_bronze = df_bronze.count()
print(f"\n✅ Total de registros lidos da Bronze: {total_bronze}")

## 2️⃣ Limpeza e Padronização

In [0]:
df_categorizado = df_bronze.withColumn(
    "ds_categoria_temp",
    F.when(
        F.lower(F.col("nome_motivo")).rlike(
            "cadastr|dados|telefone|email|vencimento|transferência|agência|cancelamento"
        ), 
        CATEGORIAS[0]
    ).when(
        F.lower(F.col("nome_motivo")).rlike(
            "cartão|bloqueio|desbloqueio|adicional|compra|contraç"
        ),
        CATEGORIAS[1]
    ).when(
        F.lower(F.col("nome_motivo")).rlike(
            "fatura|limite|contrato|renegociação|dívida|pagamento"
        ),
        CATEGORIAS[2]
    ).when(
        F.lower(F.col("nome_motivo")).rlike(
            "aplicativo|app|site|sistema|erro|problema|técnico"
        ),
        CATEGORIAS[3]
    ).when(
        F.lower(F.col("nome_motivo")).rlike(
            "pontos|programa|benefício"
        ),
        CATEGORIAS[4]
    ).otherwise(CATEGORIAS[5])
)

print("✅ Categorização automática aplicada")

In [0]:
crit_map = F.create_map(
    *[item for kv in CRITICIDADE.items() for item in (F.lit(kv[0]), F.lit(kv[1]))]
)

df_limpo = (
    df_categorizado
    .select(
        F.col("id_motivo").cast("bigint").alias("cd_motivo"),
        F.trim(F.col("nome_motivo")).alias("ds_motivo"),
        crit_map.getItem(F.upper(F.trim(F.col("criticidade")))).alias("ds_criticidade"),
        F.col("ds_categoria_temp").alias("ds_categoria"),
        # CORREÇÃO: Mantém estritamente o timestamp da origem (Bronze)
        F.col("ingestion_timestamp").alias("dt_ingestion"),
        F.coalesce(F.col("origem"), F.lit("base_motivos"))
            .alias("dc_origem"),
    )
    .dropDuplicates(["cd_motivo"])
    .withColumn("data_processamento", F.current_timestamp())
)

total_limpo = df_limpo.count()
print(f"✅ Dados limpos e padronizados: {total_limpo} registros")

## 3️⃣ Regras de Validação e Qualidade

In [0]:
df_validacao = (
    df_limpo
    # Flag 1: cd_motivo não pode ser NULL
    .withColumn(
        "flag_id_valido", 
        F.col("cd_motivo").isNotNull()
    )
    # Flag 2: ds_motivo não pode estar vazio ou NULL
    .withColumn(
        "flag_nome_valido", 
        (F.col("ds_motivo").isNotNull()) & (F.length(F.trim(F.col("ds_motivo"))) > 0)
    )
    # Flag 3: ds_categoria não pode estar vazio ou NULL
    .withColumn(
        "flag_categoria_valida", 
        (F.col("ds_categoria").isNotNull()) & (F.length(F.trim(F.col("ds_motivo"))) > 0)
    )
    # Flag 4: ds_criticidade não pode estar vazio ou NULL
    .withColumn(
        "flag_criticidade_valida", 
        (F.col("ds_criticidade").isNotNull()) & (F.length(F.trim(F.col("ds_criticidade"))) > 0)
    )
    # Flag geral de qualidade
    .withColumn(
        "flag_qualidade",
        F.when(
            F.col("flag_id_valido") &
            F.col("flag_nome_valido") &
            F.col("flag_categoria_valida") &
            F.col("flag_criticidade_valida"),
            F.lit("OK")
        ).otherwise(F.lit("ERRO"))
    )
)
print("✅ Regras de validação aplicadas")

## 4️⃣ Separação de Dados Válidos e Inválidos

In [0]:
# Define valid and invalid DataFrames
df_validos = df_validacao.filter(F.col("flag_qualidade") == "OK")

df_invalidos = (
    df_validacao
    .filter(F.col("flag_qualidade") == "ERRO")
    .withColumn(
        "motivo_rejeicao",
        F.concat_ws(
            "; ",
            F.when(~F.col("flag_id_valido"), F.lit("cd_motivo NULL")),
            F.when(~F.col("flag_nome_valido"), F.lit("ds_motivo vazio")),
            F.when(~F.col("flag_categoria_valida"), F.lit("ds_categoria vazio ou NULL")),
            F.when(~F.col("flag_criticidade_valida"), F.lit("ds_criticidade vazio ou NULL"))
        )
    )
    .withColumn("dt_validacao", F.lit(processing_timestamp))
    .drop(
        "flag_id_valido",
        "flag_nome_valido",
        "flag_categoria_valida",
        "flag_criticidade_valida",
        "data_processamento"
    )
)

total_validos = df_validos.count()
total_invalidos = df_invalidos.count()

print(f"ℹ️ Resumo: {total_validos} válidos | {total_invalidos} inválidos")

## 5️⃣ Gravação: MERGE de Registros Válidos

In [0]:
# ---------------------------
# MERGE registros válidos na tabela principal
# ---------------------------
if total_validos > 0:
    # Criar temp view
    df_validos.createOrReplaceTempView("temp_motivos_validos")
    
    print(f"🔄 Executando MERGE de {total_validos} registros válidos em {TABLE_TARGET_VALIDOS}...")
    
    # MERGE SQL
    spark.sql(f"""
        MERGE INTO {TABLE_TARGET_VALIDOS} AS target
        USING temp_motivos_validos AS source
        ON target.cd_motivo = source.cd_motivo
        
        WHEN MATCHED THEN
            UPDATE SET
                target.ds_motivo = source.ds_motivo,
                target.ds_categoria = source.ds_categoria,
                target.ds_criticidade = source.ds_criticidade,
                target.dt_ingestion = source.dt_ingestion,
                target.dc_origem = source.dc_origem
        
        WHEN NOT MATCHED THEN
            INSERT (
                cd_motivo,
                ds_motivo,
                ds_categoria,
                ds_criticidade,
                dt_ingestion,
                dc_origem
            )
            VALUES (
                source.cd_motivo,
                source.ds_motivo,
                source.ds_categoria,
                source.ds_criticidade,
                source.dt_ingestion,
                source.dc_origem
            )
    """)
    
    print(f"✅ MERGE concluído com sucesso!")
else:
    print("⚠️  Nenhum registro válido para inserir na Silver")

## 6️⃣ Gravação: APPEND de Registros Inválidos

In [0]:
# ---------------------------
# APPEND registros inválidos na tabela de auditoria
# ---------------------------
if total_invalidos > 0:
    print(f"📝 Enviando {total_invalidos} registro(s) inválido(s) para auditoria...")
    
    # APPEND (acumula histórico de rejeições)
    (df_invalidos.write
        .format("delta")
        .mode("append")
        .saveAsTable(TABLE_TARGET_INVALIDOS))
    
    print(f"✅ Registros inválidos gravados em {TABLE_TARGET_INVALIDOS}")
else:
    print("🎉 Nenhum registro inválido - 100% de qualidade!")